# Computer Vision

## Assignment - Gaussian Mixture Model (GMM)

### Install

In [1]:
%pip install comet_ml --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Comet Config

In [2]:
from comet_ml import Experiment

experiment = Experiment(project_name = "cv-gmm")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/mamello-justice/cv-gmm/f8636e8f3f83435c82a0d8d914814f4a



### Imports

In [3]:
import time
from os import path

import matplotlib.pyplot as plt

import numpy as np
import scipy

from cv_gmm_deep.datasets import load_data
from cv_gmm_deep.common import flat_images, split_data, preprocess_data
from cv_gmm_deep.gmm import GMM

print(f"numpy=={np.__version__}")
print(f"scipy=={scipy.__version__}")

c:\Users\LENOVO\.pyenv-win-venv\envs\cv_gmm_deep\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


numpy==1.23.4
scipy==1.9.3


### Constants

In [4]:
assets_dir = './assets'
data_dir = './data'

default_input_size = [768, 1024]
default_input_dir = path.join(
    assets_dir,
    f'puzzle_corners_{default_input_size[1]}x{default_input_size[0]}')

time_now = int(time.time())

default_cp_path = path.join(data_dir, str(time_now), 'cp.ckpt')
default_model_path = path.join(data_dir, str(time_now), 'model.h5')

### Params

In [5]:
params = {
    'batch_size': 3,
    'epochs': 2,
    'features': ['rgb', 'DoG'],
    'input_size': default_input_size,
}

experiment.log_parameters(params)

### Setup

In [6]:
def setup_args():
    return {
        'input_dir': default_input_dir,
        'model_path': default_model_path,
        'update_model': None
    }

### Train

In [7]:
args = setup_args()

batch_size = params['batch_size']
epochs = params['epochs']
features = params['features']
input_size = params['input_size']

input_dir = args['input_dir']
model_path = args['model_path']
update_model = args['update_model']

height, width = input_size

In [8]:
raw_x, raw_y = load_data(input_dir, height, width)

Reading in masks: 100%|██████████| 48/48 [00:00<00:00, 231.89it/s]


In [9]:
data_x, data_y = preprocess_data(raw_x, raw_y, features)

Adding DoG: 100%|██████████| 48/48 [00:10<00:00,  4.49it/s]


In [10]:
data_x, data_y = flat_images(data_x, data_y)

In [11]:
(train_x, train_y), (val_x, val_y), (test_x, test_y) =\
    split_data(data_x, data_y, ratios=[0.7, .15, .15])
    
experiment.log_dataset_hash(train_x)

In [12]:
N, pixels, features = train_x.shape

if update_model and model_path:
    model = GMM.load_model(model_path)
else:
    model = GMM(K=2, D=features, I=pixels)

assert model is not None, "Could not load model"

model.compile()

success = model.train(train_x, epochs=epochs)

output, _ = model(train_x[:2])
plt.imshow(np.reshape(output[0], (height, width)), cmap="gray")

if model_path is not None:
    model.save(model_path)


epoch 0:   3%|▎         | 1/33 [00:06<03:25,  6.43s/it]


LinAlgError: When `allow_singular is False`, the input matrix must be symmetric positive definite.

### Comet Commit

In [ ]:
experiment.end()